# VacationPy
---
## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,san clemente,33.4270,-117.6120,17.01,67,0,5.81,US,1712637445
1,1,codrington,-38.2667,141.9667,13.80,56,99,6.59,AU,1712637832
2,2,hailar,49.2000,119.7000,1.49,74,89,4.77,CN,1712637833
3,3,bredasdorp,-34.5322,20.0403,15.41,87,92,2.39,ZA,1712637834
4,4,dryden,49.7833,-92.7503,2.16,100,100,2.06,CA,1712637835


## Step 1: Create a map that displays a point for every city in the city_data_df DataFrame. The size of the point should be the humidity in each city

In [3]:
%%capture --no-display

# Configure the map plot
humidity_map = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "ESRI",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    scale =1,
    color = "City"
)

# Display the map
humidity_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

## Step 2: Narrow down the city_data_df DataFrame to find your ideal weather condition

In [4]:

# Narrow down cities that fit criteria and drop any results with null values

# My Criteria:
# 1. humidity between 30% and 60%
# 2. Max Temp = 15 and over


myConditions_df = city_data_df[(city_data_df["Humidity"] >= 30) & 
                               (city_data_df["Humidity"] <= 60) & 
                               (city_data_df["Max Temp"] >= 15) 
                               ]

# Drop any rows with null values
myConditions_df.dropna()

# Display sample data
myConditions_df


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
8,8,bamboo flat,11.7000,92.7167,33.05,58,40,2.57,IN,1712637838
20,20,ca mau,9.1769,105.1500,37.86,31,99,6.06,VN,1712637621
46,46,tera,14.0078,0.7531,29.85,36,93,4.22,NE,1712637608
47,47,tame,6.4610,-71.7300,26.74,52,78,2.12,CO,1712637873
53,53,cheremushki,52.8356,91.3944,15.94,50,74,1.12,RU,1712637879
...,...,...,...,...,...,...,...,...,...,...
549,549,niafunke,15.9322,-3.9906,30.09,30,46,4.53,ML,1712638554
567,567,kattivakkam,13.2167,80.3167,32.28,56,79,4.85,IN,1712638573
570,570,myeik,12.4333,98.6000,32.65,58,85,2.38,MM,1712638576
572,572,lhuentse,27.6679,91.1839,20.49,40,87,1.85,BT,1712638577


## Create a new DataFrame called hotel_df

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
new_Data= { "City": myConditions_df["City"],
            "Country": myConditions_df["Country"],
            "Latitude": myConditions_df["Lat"],
            "Longitude": myConditions_df["Lng"],
            "Humidity": myConditions_df["Humidity"]}
hotel_df = pd.DataFrame(new_Data)
# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""
# hotel = hotel_df["Hotel Name"]

# Display sample data
hotel_df

,City,Country,Latitude,Longitude,Humidity,Hotel Name
8,bamboo flat,IN,11.7000,92.7167,58,
20,ca mau,VN,9.1769,105.1500,31,
46,tera,NE,14.0078,0.7531,36,
47,tame,CO,6.4610,-71.7300,52,
53,cheremushki,RU,52.8356,91.3944,50,
...,...,...,...,...,...,...
549,niafunke,ML,15.9322,-3.9906,30,
567,kattivakkam,IN,13.2167,80.3167,56,
570,myeik,MM,12.4333,98.6000,58,
572,lhuentse,BT,27.6679,91.1839,40,


## Step 4: For each city, use the Geoapify APY to find the first hotel located within 10,000 metres of your coordinates

In [6]:
# Set parameters to search for a hotel
radius = 10000
long = []
lat = []
categories = "accommodation.hotel"
# filters = f"circle: {long},{lat},{radius}"
# bias = f"proximity: {long},{lat}"
limit = 10
params = {
    "categories":categories,
    "limit":limit,
    # "filter": filters,
    # "bias": bias,
    "apiKey":geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = hotel_df.loc[index,"Latitude"]
    long = hotel_df.loc[index,"Longitude"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{long},{lat},{radius}"
    params["bias"] = f"proximity:{long},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params = params)
    
    # Convert the API response to JSON format
    name_address =  name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
bamboo flat - nearest hotel: Hotel Megapode Nest
ca mau - nearest hotel: Khách sạn Song Ngọc
tera - nearest hotel: Hôtel Daressalam Téra
tame - nearest hotel: hotel libertadores
cheremushki - nearest hotel: Борус
blackmans bay - nearest hotel: Villa Howden
eyl - nearest hotel: فندق ايل دابي
fort bragg - nearest hotel: Airborne Inn Lodging
roven'ky - nearest hotel: No hotel found
karratha - nearest hotel: Karratha International Hotel
karaton - nearest hotel: Orange Syariah
mangrol - nearest hotel: No hotel found
al jawf - nearest hotel: No hotel found
jawf al maqbabah - nearest hotel: فندق لودر
al muwayh - nearest hotel: فندق ومطعم الموية
banda aceh - nearest hotel: Hotel 61
laguna - nearest hotel: Holiday Inn Express & Suites
margaret river - nearest hotel: Margaret River Hotel
napier - nearest hotel: Scenic Hotel Te Pania
isla vista - nearest hotel: The Club & Guest House
ghormach - nearest hotel: No hotel found
kiziltepe - nearest hotel: Dunaysır Otel
gargaliano

,City,Country,Latitude,Longitude,Humidity,Hotel Name
8,bamboo flat,IN,11.7000,92.7167,58,Hotel Megapode Nest
20,ca mau,VN,9.1769,105.1500,31,Khách sạn Song Ngọc
46,tera,NE,14.0078,0.7531,36,Hôtel Daressalam Téra
47,tame,CO,6.4610,-71.7300,52,hotel libertadores
53,cheremushki,RU,52.8356,91.3944,50,Борус
...,...,...,...,...,...,...
549,niafunke,ML,15.9322,-3.9906,30,No hotel found
567,kattivakkam,IN,13.2167,80.3167,56,No hotel found
570,myeik,MM,12.4333,98.6000,58,Green Eyes Hotel
572,lhuentse,BT,27.6679,91.1839,40,Hotel Shangrila


## Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map

In [7]:
%%capture --no-display

# Configure the map plot
hotel_map = hotel_df.hvplot.points(
    "Longitude",
    "Latitude",
    geo = True,
    tiles = "ESRI",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    scale = 1,
    color = "City",
    hover_cols = ["Country","Hotel Name"]
)

# Display the map
hotel_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Longitude,Latitude]   (City,Humidity,Country,Hotel Name)